In [83]:
import pandas as pd
train = pd.read_csv('train_python.csv')
test = pd.read_csv('test_python.csv')

train.head()

,max_floor,state,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,...,avg_open_to_buy,build_year,incineration_raion,full_sq,total_relationship_count,detention_facility_raion,build_count_mix,railroad_terminal_raion,__churn,__price_doc
0,NaN,NaN,Single,no,1906,1,0,0.694,no,5.419893,...,1996.9,NaN,no,43,5,no,0.0,no,0,5.850000
1,NaN,NaN,Married,no,12,0,6,0.693,no,3.411993,...,1338.4,NaN,no,34,3,no,0.0,no,0,6.000000
2,NaN,NaN,Married,no,2153,5,0,0.831,no,1.277658,...,2411.5,NaN,no,43,6,no,0.0,no,0,5.700000
3,NaN,NaN,Married,no,1806,2,0,0.638,no,4.291432,...,9255.9,NaN,no,89,3,no,2.0,no,0,13.100000
4,NaN,NaN,Married,no,753,2,40,0.902,no,0.853960,...,3318.2,NaN,no,77,4,no,1.0,yes,0,16.331452


# Removing outliers

Удалим из числовых данных выбросы с помощью Quiantile Transformation

In [84]:
from sklearn.preprocessing import QuantileTransformer

for i in train.select_dtypes(include = ['int64', 'float64']).columns:
    if i not in['__churn', '__price_doc']: #для всех переменных кроме предсказываемых!!! убираем выбросы
        quntile = QuantileTransformer(output_distribution = 'normal')
        quntile.fit(train[[i]])
        
        train[f'f_{i}'] = quntile.transform(train[[i]]) #создаем доп переменные с именами f_старое имя
        test[f'f_{i}'] = quntile.transform(test[[i]])

# Refilling nulls with medians

Посмотрим, какие переменные содержат Null

In [85]:
result = (
    train
    .isnull().sum()                   # count of non-null values
    .sort_values(ascending = False)     # sort values using descending order
)

result[result > 0]

f_state                          12014
state                            12014
f_build_year                     11582
build_year                       11582
max_floor                         9572
f_num_room                        9572
f_max_floor                       9572
num_room                          9572
f_cafe_sum_1000_min_price_avg     4596
cafe_sum_1000_min_price_avg       4596
f_life_sq                         4193
life_sq                           4193
build_count_wood                  3063
f_build_count_wood                3063
build_count_mix                   3063
f_build_count_mix                 3063
cafe_sum_1500_min_price_avg       2998
f_cafe_sum_1500_min_price_avg     2998
f_total_trans_amt                 2403
0_17_all                          2403
f_0_17_all                        2403
total_trans_amt                   2403
floor                              167
f_floor                            167
railroad_station_walk_km            10
metro_km_walk            

Заполним пропущенные значения медианами

In [86]:
for i in result[result > 0].index:
    train_modes = train.dropna().median() #считаем медианы для данных без null
    train[i].fillna(train_modes[i], inplace = True)
    
    test[i].fillna(train_modes[i], inplace = True) #заполняем в тесте пропуски медианными данными из трейна!!!

    
result1 = train.isnull().sum().sort_values()
result1[result1 > 0]

result2 = test.isnull().sum().sort_values()
result2[result2 > 0]

C:\Users\User\AppData\Local\Temp\ipykernel_7644\1403204896.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_modes = train.dropna().median() #считаем медианы для данных без null


Series([], dtype: int64)

# Target Encoder

Target Encoder: 

Числовые переменные разбиваем на бины с одинаковым количеством наблюдений. Каждому бину присваиваем среднее значение по price и churn!!! (по таргетной переменной)

То есть у нас теперь для всех наблюдений в одном бине одно значение - среднее по таргетной переменной для этого бина.


Потом эти переменные передаем в регрессию, и как бы сама регрессия линейная, но переменная не совсем линейная.
Переменная становится более резкой, и так модель лучше будет работать. Можно поэкспериментировать с количеством бинов, но в целом, чем больше данных, тем больше должно быть бинов


Для категориальных можно как Target Encoder, так и One-Hot. Для числовых - Target Hot, Standard Scaler, Robust Scaler (лучше все вместе пхпх)

In [ ]:
pip install category_encoders

In [6]:
#посмотрим для одной переменной сначала как выглядят эти бины, потом реализуем для всех ньюмерик переменных
num_quantiles = 10
_, bins_try = pd.qcut(train['max_floor'],
                  q = num_quantiles,
                  retbins = True, duplicates = 'drop') #важно добавить drop потому что некоторые бины имеют одну границу и код ломается
bins_try

array([ 0.,  5.,  9., 12., 16., 17., 48.])

In [87]:
from category_encoders import TargetEncoder

for i in train.select_dtypes(include = ['int64', 'float64']).columns:
    if i not in ['__churn', '__price_doc']:
        
        #сделаем Target Encoding только на преобразованных переменных, без выбросов
        if i not in ['max_floor', 'state', 'total_revolving_bal', 'market_count_1500',
       'leisure_count_3000', 'total_ct_chng_q4_q1', 'railroad_station_walk_km',
       'contacts_count_12_mon', '0_17_all', 'trc_count_2000',
       'build_count_wood', 'credit_limit', 'total_trans_ct',
       'leisure_count_5000', 'life_sq', 'cafe_count_1000_price_1000',
       'mkad_km', 'school_education_centers_top_20_raion',
       'avg_utilization_ratio', 'public_transport_station_min_walk',
       'customer_age', 'detention_facility_km', 'sport_count_2000',
       'cafe_sum_1000_min_price_avg', 'total_amt_chng_q4_q1', 'metro_km_walk',
       'office_sqm_5000', 'total_trans_amt', 'months_inactive_12_mon',
       'cafe_sum_1500_min_price_avg', 'floor', 'num_room', 'months_on_book',
       'dependent_count', 'avg_open_to_buy', 'build_year', 'full_sq',
       'total_relationship_count', 'build_count_mix']:
            
            num_quantiles = 10
            _, bins = pd.qcut(train[i],
                              q = num_quantiles,
                              retbins = True, duplicates = 'drop') #разделили на границы бинов
            train[f'boards_{i}'] = pd.cut(train[i], bins = bins).astype(str)
            test[f'boards_{i}'] = pd.cut(test[i], bins = bins).astype(str)
            
            enc = TargetEncoder()
            enc.fit(train[f'boards_{i}'], train['__price_doc']) #посчитали среднее по price_doc

            train[f'PRICE_{i}'] = enc.transform(train[f'boards_{i}'])
            test[f'PRICE_{i}'] = enc.transform(test[f'boards_{i}'])


            enc = TargetEncoder()
            enc.fit(train[f'boards_{i}'], train['__churn']) #посчитали среднее по churn

            train[f'CHURN_{i}'] = enc.transform(train[f'boards_{i}'])
            test[f'CHURN_{i}'] = enc.transform(test[f'boards_{i}'])
        
#сделали Target Encoding для всех numeric переменных. Сначала разделили каждую на 10 бинов
#потом посчитали среднее значение целевой переменной для каждого бина

C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[f'boards_{i}'] = pd.cut(test[i], bins = bins).astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f'PRICE_{i}'] = enc.transform(train[f'boards_{i}'])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f'CHURN_{i}'] = enc.transform(train[f'boards_{i}'])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[f'CHURN_{i}'] = enc.transform(test[f'boards_{i}'])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\3240603760.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

In [88]:
#чтобы не было 100 переменных уалим  вспомогатльные, которые содержат слово boards

train = train[train.columns.drop(list(train.filter(regex = 'boards')))]
test = test[test.columns.drop(list(test.filter(regex = 'boards')))]

# Z-score standardization

Стандартизуем переменные, чтобы они имели распределение от -1 до 4

In [89]:
from sklearn.preprocessing import StandardScaler

for i in train.select_dtypes(include = ['int64', 'float64']).columns:
    if i not in ['__churn', '__price_doc']:
        
        if i not in ['max_floor', 'state', 'total_revolving_bal', 'market_count_1500',
       'leisure_count_3000', 'total_ct_chng_q4_q1', 'railroad_station_walk_km',
       'contacts_count_12_mon', '0_17_all', 'trc_count_2000',
       'build_count_wood', 'credit_limit', 'total_trans_ct',
       'leisure_count_5000', 'life_sq', 'cafe_count_1000_price_1000',
       'mkad_km', 'school_education_centers_top_20_raion',
       'avg_utilization_ratio', 'public_transport_station_min_walk',
       'customer_age', 'detention_facility_km', 'sport_count_2000',
       'cafe_sum_1000_min_price_avg', 'total_amt_chng_q4_q1', 'metro_km_walk',
       'office_sqm_5000', 'total_trans_amt', 'months_inactive_12_mon',
       'cafe_sum_1500_min_price_avg', 'floor', 'num_room', 'months_on_book',
       'dependent_count', 'avg_open_to_buy', 'build_year', 'full_sq',
       'total_relationship_count', 'build_count_mix']:
                     
            tr_sc = StandardScaler()
            tr_sc.fit(train[[i]])

            train[f'Zscore_{i}'] = tr_sc.transform(train[[i]])
            test[f'Zscore_{i}'] = tr_sc.transform(test[[i]])

C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[f'Zscore_{i}'] = tr_sc.transform(train[[i]])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'Zscore_{i}'] = tr_sc.transform(test[[i]])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'Zscore_{i}'] = tr_sc.transform(test[[i]])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'Zscore_{i}'] = tr_sc.transform(test[[i]])
C:\Users\User\AppData\Local\Temp\ipykernel_7644\1338323512.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

# One-Hot Encoder

One-Hot Encoder - для категориальных переменных используем

Он трансформирует все категориальные переменные в дамми по сути

In [ ]:
pip install --upgrade sklearn.preprocessing

In [90]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse = False, handle_unknown='ignore')

for i in train.select_dtypes(include = ['object']).columns:
    if i != 'timestamp':
        onehot.fit(train[[i]])
        values_train = onehot.transform(train[[i]])
        values_test = onehot.transform(test[[i]])
        list_names = []
        
        for k, cat in enumerate(onehot.categories_[0]):
            name = f'f_{i}_{cat}'
            list_names.append(name)
            train[name] = values_train[:, k]
            test[name] = values_test[:, k]

In [91]:
train.head()

,max_floor,state,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,...,f_railroad_1line_no,f_railroad_1line_yes,f_education_level_Graduate,f_education_level_High School,f_incineration_raion_no,f_incineration_raion_yes,f_detention_facility_raion_no,f_detention_facility_raion_yes,f_railroad_terminal_raion_no,f_railroad_terminal_raion_yes
0,12.0,2.0,Single,no,1906,1,0,0.694,no,5.419893,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,12.0,2.0,Married,no,12,0,6,0.693,no,3.411993,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,12.0,2.0,Married,no,2153,5,0,0.831,no,1.277658,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,12.0,2.0,Married,no,1806,2,0,0.638,no,4.291432,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,12.0,2.0,Married,no,753,2,40,0.902,no,0.853960,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
